In [1]:
from hypex.dataset.dataset import Dataset, ExperimentData
from hypex.experiments.aa_test import AA_TEST
from hypex.dataset.roles import (
    InfoRole,
    FeatureRole,
    TreatmentRole,
    PreTargetRole,
    TargetRole
)

In [2]:
data = Dataset(
    "data.csv", 
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": PreTargetRole(),
        "post_spends": TargetRole(),
        "age": FeatureRole(),
        "gender": FeatureRole(),
        "industry":  FeatureRole()
    }
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': <hypex.dataset.roles.InfoRole at 0x19d1c3571c0>,
 'signup_month': <hypex.dataset.roles.FeatureRole at 0x19d5bf7d210>,
 'treat': <hypex.dataset.roles.TreatmentRole at 0x19d5bf7cc40>,
 'pre_spends': <hypex.dataset.roles.PreTargetRole at 0x19d5bf7cca0>,
 'post_spends': <hypex.dataset.roles.TargetRole at 0x19d5bf7cd00>,
 'age': <hypex.dataset.roles.FeatureRole at 0x19d5bf7d540>,
 'gender': <hypex.dataset.roles.FeatureRole at 0x19d5bf7d720>,
 'industry': <hypex.dataset.roles.FeatureRole at 0x19d5bf7d630>}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

AttributeError: 'ExperimentData' object has no attribute '_set_value'

In [18]:
import numpy as np

In [7]:
ed.data.data.sample(frac=1, random_state=1)

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
1820,1820,4,1,489.5,502.555556,NaN,M,E-commerce
2127,2127,0,0,485.5,418.888889,64.0,F,Logistics
4058,4058,0,0,458.5,416.000000,59.0,M,E-commerce
7222,7222,0,0,483.5,420.222222,29.0,M,Logistics
3000,3000,1,1,512.5,523.444444,NaN,F,Logistics
...,...,...,...,...,...,...,...,...
3459,3459,0,0,495.5,417.888889,60.0,M,E-commerce
7359,7359,11,1,460.5,435.777778,28.0,F,E-commerce
2383,2383,0,0,509.0,422.777778,32.0,M,Logistics
9147,9147,0,0,507.0,433.444444,28.0,M,Logistics
